In [1]:
!git clone https://github.com/motazsaad/Quran-QA

!git clone https://gitlab.com/bigirqu/quranqa.git
!pip install git+https://github.com/neuml/txtai
!pip install farasapy

Cloning into 'Quran-QA'...
remote: Enumerating objects: 116, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 116 (delta 46), reused 74 (delta 18), pack-reused 0
Receiving objects: 100% (116/116), 1.14 MiB | 5.57 MiB/s, done.
Resolving deltas: 100% (46/46), done.
Cloning into 'quranqa'...
remote: Enumerating objects: 333, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 333 (delta 43), reused 86 (delta 43), pack-reused 247
Receiving objects: 100% (333/333), 312.88 KiB | 11.59 MiB/s, done.
Resolving deltas: 100% (130/130), done.
  Cloning https://github.com/neuml/txtai to /tmp/pip-req-build-tgq47j1p
  Running command git clone --filter=blob:none --quiet https://github.com/neuml/txtai /tmp/pip-req-build-tgq47j1p
  Resolved https://github.com/neuml/txtai to commit 12646f2bc277067efd983bd2d9783a733d8f2aac
  Installing build dependencies ... done
  Getting re

In [2]:
import sys
sys.path.insert(1, './quranqa/code')

from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from txtai.pipeline import HFTrainer
import torch, string, re, os, json
import operator
import numpy as np


from farasa.pos import FarasaPOSTagger
from farasa.ner import FarasaNamedEntityRecognizer
from farasa.diacratizer import FarasaDiacritizer
from farasa.segmenter import FarasaSegmenter
from farasa.stemmer import FarasaStemmer

In [3]:
import json ,argparse

In [4]:
def json_load(input_path)->list:
  data=[]
  with open(input_path,'r',encoding='utf-8') as f:
    for line in f:
      data.append(json.load(line.restrip('\n|\r')))
  print('Loaded{} records from {}'.format(len(data),input_path))
  return data

In [6]:
import pandas as pd
import json

train_file = '/content/Quran-QA/quranqa/datasets/qrcd_v1.1_train.jsonl'
dev_file = '/content/quranqa/datasets/qrcd_v1.1_dev.jsonl'
test_file='/content/quranqa/datasets/Full_test_dataset/qrcd_v1.1_test.jsonl'
# Function to load and normalize JSONL file
def load_jsonl_to_df(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        data = [json.loads(line) for line in file]
    return pd.json_normalize(data)

# Load training and development data
df_train = load_jsonl_to_df(train_file)
df_dev = load_jsonl_to_df(dev_file)
df_test=load_jsonl_to_df(test_file)
# Display the first few rows of the training dataframe
len(df_train)


710

In [7]:
len(df_dev)

109

In [8]:
len(df_test)

274

In [9]:
df_combined_train = pd.concat([df_train, df_test])
df_combined_train.reset_index(drop=True, inplace=True)
df_combined_train.head()


,pq_id,passage,surah,verses,question,answers
0,2:8-16_364,ومن الناس من يقول آمنا بالله وباليوم الآخر وما...,2,8-16,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,"[{'text': 'أولئك الذين اشتروا الضلالة بالهدى',..."
1,2:174-176_364,إن الذين يكتمون ما أنزل الله من الكتاب ويشترون...,2,174-176,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,[{'text': 'أولئك الذين اشتروا الضلالة بالهدى و...
2,14:47-52_364,فلا تحسبن الله مخلف وعده رسله إن الله عزيز ذو ...,14,47-52,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,"[{'text': 'ليجزي الله كل نفس ما كسبت', 'start_..."
3,17:12-17_364,وجعلنا الليل والنهار آيتين فمحونا آية الليل وج...,17,12-17,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,"[{'text': 'كل إنسان ألزمناه طائره في عنقه', 's..."
4,18:29-31_364,وقل الحق من ربكم فمن شاء فليؤمن ومن شاء فليكفر...,18,29-31,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,"[{'text': 'من شاء فليؤمن ومن شاء فليكفر', 'sta..."


In [10]:
def add_end_idx(df):
  for idx,row in df.iterrows():
    passege=row["passage"]
    for answer in row["answers"]:
      best_text=answer["text"]
      start_idx=answer["start_char"]
      end_idx=start_idx+len(best_text)

      if passege[start_idx:end_idx]==best_text:
        answer["end_char"] =end_idx
      else:
        for n in [1,2]:
          if passege[start_idx-n:end_idx-n]==best_text:
            answer['start_char']=start_idx-n
            answer['end_char']=end_idx-n

add_end_idx(df_combined_train)
add_end_idx(df_dev)

In [11]:
add_end_idx(df_test)

In [12]:
df_combined_train.head()

,pq_id,passage,surah,verses,question,answers
0,2:8-16_364,ومن الناس من يقول آمنا بالله وباليوم الآخر وما...,2,8-16,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,"[{'text': 'أولئك الذين اشتروا الضلالة بالهدى',..."
1,2:174-176_364,إن الذين يكتمون ما أنزل الله من الكتاب ويشترون...,2,174-176,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,[{'text': 'أولئك الذين اشتروا الضلالة بالهدى و...
2,14:47-52_364,فلا تحسبن الله مخلف وعده رسله إن الله عزيز ذو ...,14,47-52,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,"[{'text': 'ليجزي الله كل نفس ما كسبت', 'start_..."
3,17:12-17_364,وجعلنا الليل والنهار آيتين فمحونا آية الليل وج...,17,12-17,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,"[{'text': 'كل إنسان ألزمناه طائره في عنقه', 's..."
4,18:29-31_364,وقل الحق من ربكم فمن شاء فليؤمن ومن شاء فليكفر...,18,29-31,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,"[{'text': 'من شاء فليؤمن ومن شاء فليكفر', 'sta..."


In [13]:
df_dev.head()

,pq_id,passage,surah,verses,question,answers
0,9:60-61_316,إنما الصدقات للفقراء والمساكين والعاملين عليها...,9,60-61,ما هي مصارف الزكاة؟,[{'text': 'للفقراء والمساكين والعاملين عليها و...
1,2:178-179_400,يا أيها الذين آمنوا كتب عليكم القصاص في القتلى...,2,178-179,متى يحل الإسلام دم الشخص؟,[{'text': 'كتب عليكم القصاص في القتلى الحر بال...
2,2:190-194_400,وقاتلوا في سبيل الله الذين يقاتلونكم ولا تعتدو...,2,190-194,متى يحل الإسلام دم الشخص؟,[{'text': 'قاتلوا في سبيل الله الذين يقاتلونكم...
3,2:243-245_400,ألم تر إلى الذين خرجوا من ديارهم وهم ألوف حذر ...,2,243-245,متى يحل الإسلام دم الشخص؟,"[{'text': 'قاتلوا في سبيل الله', 'start_char':..."
4,4:74-76_400,فليقاتل في سبيل الله الذين يشرون الحياة الدنيا...,4,74-76,متى يحل الإسلام دم الشخص؟,"[{'text': 'الذين آمنوا يقاتلون في سبيل الله', ..."


In [14]:
from transformers import AutoTokenizer
import pandas as pd

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("wissamantoun/araelectra-base-artydiqa")

# Function to extract contexts, questions, and answers
def extract_qa_pairs(df):
    contexts = df['passage'].tolist()
    questions = df['question'].tolist()
    answers = df['answers'].tolist()
    return contexts, questions, answers

# Extract data from training and validation DataFrames
train_contexts, train_questions, train_answers = extract_qa_pairs(df_combined_train)
val_contexts, val_questions, val_answers = extract_qa_pairs(df_dev)

# Tokenize the data
train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        # Get the first answer in the list of answers for this question
        answer = answers[i][0]

        # Adjust start and end positions for token offsets
        start_positions.append(encodings.char_to_token(i, answer['start_char']))
        end_positions.append(encodings.char_to_token(i, answer['end_char'] - 1))

        # Handle potential None values
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length

        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length - 1

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})


# Apply the function to your tokenized data
add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)


tokenizer_config.json:   0%|          | 0.00/390 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/771 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/761k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [15]:
len(train_encodings["start_positions"])


984

In [16]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'])

In [17]:
import torch

class QuranQA(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

# build datasets for both our training and validation sets
train_dataset = QuranQA(train_encodings)
val_dataset = QuranQA(val_encodings)

In [18]:
import torch
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR

# Assume train_encodings and val_encodings are already prepared
# train_dataset = QuranQA(train_encodings)
# val_dataset = QuranQA(val_encodings)
model_name="wissamantoun/araelectra-base-artydiqa"
# Load your pre-trained model here (e.g., BERT for question answering)
model =AutoModelForQuestionAnswering.from_pretrained(model_name)

# Set up GPU/CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Move model to the detected device
model.to(device)

# Activate training mode of the model
model.train()

# Initialize the AdamW optimizer with weight decay
optim = AdamW(model.parameters(), lr=1e-5)

# Initialize the DataLoader for the training data
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Training loop
for epoch in range(11):  # Number of epochs
    # Set model to training mode
    model.train()

    # Set up the progress bar
    loop = tqdm(train_loader, leave=True)

    for batch in loop:

        # Initialize calculated gradients (from the previous step)
        optim.zero_grad()

        # Pull all tensor batches required for training and move them to the device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)

        # Train model on batch and return outputs (including loss)
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)

        # Extract loss
        loss = outputs.loss

        # Calculate loss for every parameter with grad update
        loss.backward()

        # Update parameters
        optim.step()

        # Update the progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())



model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 10: 100%|██████████| 62/62 [01:31<00:00,  1.48s/it, loss=0.0369]


In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR

# Assume train_encodings and val_encodings are already prepared
# train_dataset = QuranQA(train_encodings)
# val_dataset = QuranQA(val_encodings)
model_name="wissamantoun/araelectra-base-artydiqa"
# Load your pre-trained model here (e.g., BERT for question answering)
model =AutoModelForQuestionAnswering.from_pretrained(model_name)

# Set up GPU/CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Move model to the detected device
model.to(device)

# Activate training mode of the model
model.train()

# Initialize the AdamW optimizer with weight decay
optim = AdamW(model.parameters(), lr=5e-5)

# Initialize the DataLoader for the training data
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Training loop
for epoch in range(15):  # Number of epochs
    # Set model to training mode
    model.train()

    # Set up the progress bar
    loop = tqdm(train_loader, leave=True)

    for batch in loop:

        # Initialize calculated gradients (from the previous step)
        optim.zero_grad()

        # Pull all tensor batches required for training and move them to the device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)

        # Train model on batch and return outputs (including loss)
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)

        # Extract loss
        loss = outputs.loss

        # Calculate loss for every parameter with grad update
        loss.backward()

        # Update parameters
        optim.step()

        # Update the progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 6:  10%|▉         | 6/62 [00:08<01:23,  1.48s/it, loss=0.213]

In [19]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score

model.eval()
val_loader = DataLoader(val_dataset, batch_size=16)

# Lists to store true positives, false positives, and false negatives
tp_start, fp_start, fn_start = [], [], []
tp_end, fp_end, fn_end = [], [], []

for batch in val_loader:
    with torch.no_grad():
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_true = batch['start_positions'].to(device)
        end_true = batch['end_positions'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        start_pred = torch.argmax(outputs['start_logits'], dim=1)
        end_pred = torch.argmax(outputs['end_logits'], dim=1)

        # Calculate true positives, false positives, and false negatives for start
        tp_start.append(((start_pred == start_true) & (start_pred != 0)).sum().item())
        fp_start.append(((start_pred != start_true) & (start_pred != 0)).sum().item())
        fn_start.append(((start_pred != start_true) & (start_true != 0)).sum().item())

        # Calculate true positives, false positives, and false negatives for end
        tp_end.append(((end_pred == end_true) & (end_pred != 0)).sum().item())
        fp_end.append(((end_pred != end_true) & (end_pred != 0)).sum().item())
        fn_end.append(((end_pred != end_true) & (end_true != 0)).sum().item())

# Calculate precision, recall, and F1 score
precision_start = sum(tp_start) / (sum(tp_start) + sum(fp_start))
recall_start = sum(tp_start) / (sum(tp_start) + sum(fn_start))
f1_start = 2 * (precision_start * recall_start) / (precision_start + recall_start)

precision_end = sum(tp_end) / (sum(tp_end) + sum(fp_end))
recall_end = sum(tp_end) / (sum(tp_end) + sum(fn_end))
f1_end = 2 * (precision_end * recall_end) / (precision_end + recall_end)

print("F1 Score for Start Positions:", f1_start)
print("F1 Score for End Positions:", f1_end)


F1 Score for Start Positions: 0.44954128440366975
F1 Score for End Positions: 0.44036697247706424
